## Notebook principal

Primero se busca trabajar con 3 libros de texto en formato PDF, los cuales son:  
Codigo limpio - Robert Cecil Martin  
Ingenieria de software - Pressman  
Ingenieria de software - Sommerville

In [ ]:
%pip install PyPDF2
%pip install tqdm

In [ ]:
from PyPDF2 import PdfReader
from tqdm.auto import tqdm
from os import makedirs, listdir, path, getcwd

BASE_DIR = getcwd()
DATA_DIR = path.join(BASE_DIR, "data")
PDF_DIR = path.join(DATA_DIR, "pdf")
TXT_DIR = path.join(DATA_DIR, "txt")

Una vez teniendo los libros en formato PDF, se procede a convertirlos a formato TXT, para poder trabajar con ellos.
La conversion ocurre pagina por pagina.

In [ ]:
makedirs(TXT_DIR, exist_ok=True)
# obtener todos los archivos pdf dentro de PDF_DIR
pdf_files = [f for f in listdir(PDF_DIR) if f.lower().endswith('.pdf')]
for pdf in pdf_files:
    # leer el archivo pdf
    with open(path.join(PDF_DIR, pdf), 'rb') as f:
        pdf_reader = PdfReader(f)
        # leer el número de páginas en el archivo pdf
        pbar = tqdm(range(len(pdf_reader.pages)))
        # iterar sobre las páginas en el archivo pdf
        page_num = 0
        for page in pbar:
            pbar.set_description(f'Processing {pdf}')
            # obtener el texto de la página eliminando los saltos de línea
            text = pdf_reader.pages[page].extract_text().replace('\n', ' ')
            # escribir el texto en un archivo txt
            txt_file = f'{pdf[:-4]}_{page_num}.txt'
            # generar un archivo txt que contenga el texto de todos los archivos txt de un mismo pdf
            main_txt_file = f'{pdf[:-4]}.txt'
            with open(path.join(TXT_DIR, txt_file), 'w', encoding='utf-8') as f, open(path.join(TXT_DIR, main_txt_file), 'a', encoding='utf-8') as mwf:
                # formatear el texto para normalizarlo
                formatted_text = text.replace('\n', ' ').lower().replace('  ', ' ')
                f.write(formatted_text)
                mwf.write(formatted_text)
                page_num += 1

Una vez teniendo todos los archivos .txt se debe obtener el dataset de STS lleno de oraciones relacionadas a la ingenieria de software y un puntaje entre 0 y 1 que valore la similitud entre las oraciones.

Se buscara hacer uso de algun modelo para extraer las oraciones del archivo .txt

In [ ]:
%pip install huggingface_hub transformers datasets  

Buscando los modelos en espanol para extraer automaticamente las oraciones de ingenieria de software encontre este modelo: [hiiamsid/sentence_similarity_spanish_es](https://huggingface.co/hiiamsid/sentence_similarity_spanish_es) el cual potencialmente sera usado para realizarle fine-tuning y asi obtener un modelo final.

Se busca obtener cada oracion del archivo .txt principal de cada libro e incluirla en un archivo .json.

In [ ]:
import re

def remove_useless_text(text):
    # Regular expressions for detecting useless text patterns
    patterns = [
        r'>',  # Remove bullet point indicators
        r'\d+\w*',  # Remove numbers or digits combined with letters
        r'\b\w\b',  # Remove single-letter words
        r'\b\w{1,2}\b',  # Remove words with one or two characters
        r'www\.[^\s]+',  # Remove URLs
    ]

    # Apply each pattern sequentially to remove useless text
    for pattern in patterns:
        text = re.sub(pattern, '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [ ]:
for filename in listdir(TXT_DIR):
    if not any(char.isdigit() for char in filename):
        # por la convencion de nombres que se tomo, estos archivos son los que contienen todo el texto de su respectivo pdf
        with open(TXT_DIR + '/' + filename, 'r', encoding='utf-8') as file:
            text = file.read()
            cleaned_text = remove_useless_text(text)
            # Do something with the cleaned text, such as write it to a new file
            with open(TXT_DIR + '/cleaned_' + filename, 'w', encoding='utf-8') as cleaned_file:
                cleaned_file.write(cleaned_text)

Se opta por usar los embeddings del modelo anteriormente mencionado para asi encontrar el contexto de cada oracion.